In [1]:
import pandas as pd
import gmaps
from config import gkey
import ipywidgets as widgets
from IPython.display import display

In [2]:
gmaps.configure(api_key=gkey)

In [3]:
df = pd.read_csv('species_complete.csv')

In [4]:
taxa_df = pd.DataFrame({'Level':['Genus','Family','Order','Subclass','Class','Phylum','Kingdom']})

In [5]:
def unique_sorted_values(array):
    unique = array.unique().tolist()
    unique.sort()
    unique.insert(0, 'Select species')
    return unique

def unique_values(array):
    unique = array.unique().tolist()
    return unique

In [26]:
output = widgets.Output()
plot_output = widgets.Output()

dropdown_sp = widgets.Dropdown(options = unique_sorted_values(df.ScientificName), description='Species: ')
dropdown_taxa = widgets.Dropdown(options = unique_values(taxa_df.Level), description='Level of taxonomy to compare: ')

def common_filtering(sp, tax):
    output.clear_output()
    plot_output.clear_output()
    
    common_filter = (df[df.ScientificName == sp])
    with output:
        display(common_filter)
        
    with plot_output:
        species_tag = sp
        marker_locations = df[['latitude','longitude']][df.ScientificName==sp]
        latitude = df['latitude'][df.ScientificName==sp].tolist()
        longitude = df['longitude'][df.ScientificName==sp].tolist()
        depth = df['DepthInMeters'][df.ScientificName==sp].tolist()
        locality = df['Locality'][df.ScientificName==sp].tolist()
        category = df['VernacularNameCategory'][df.ScientificName==sp].tolist()
        
        taxa_level_list = ['Kingdom','Phylum','Class','Subclass','Order','Family','Genus']
        t = taxa_level_list.index(tax) + 3
        taxa_name_to_look = df.iloc[:,t][df.ScientificName==sp].tolist()[0]
        print(f'{tax}: {taxa_name_to_look}')
        
        box_dic=[]
        for i in range(len(marker_locations)):
            box_dic.append({'sp':sp,'lat':latitude[i],'lon':longitude[i],
                            'depth':depth[i], 'loc':locality[i], 'cat':category[i]})
        info_box_template = """<div style='float:left'>
        <img src='https://upload.wikimedia.org/wikipedia/commons/thumb/2/20/Aplysina_archeri_%28Stove-pipe_Sponge-pink_variation%29.jpg/179px-Aplysina_archeri_%28Stove-pipe_Sponge-pink_variation%29.jpg'></div>
        <div style='float:right; padding: 10px;'>
        <b>Species: </b>{sp}</br>
        <b>Latitude, longitude: </b>{lat}, {lon}</br>
        <b>Depth: </b>{depth}m</br>
        <b>Locality: </b>{loc}</br>
        <b>Category: </b>{cat}
        </div>"""
        
        sp_info = [info_box_template.format(**sp) for sp in box_dic]
        locations = df.iloc[:,11:13][df.iloc[:,t]==taxa_name_to_look] 
        
        fig = gmaps.figure(map_type="HYBRID")
        markers = gmaps.marker_layer(marker_locations, info_box_content=sp_info)
        heat = gmaps.heatmap_layer(locations, dissipating=False, max_intensity=100, point_radius = 1)
        fig.add_layer(markers)
        fig.add_layer(heat)
        display(fig)
        
def dropdown_sp_eventhandler(change):
    common_filtering(change.new, dropdown_taxa.value)

def dropdown_taxa_eventhandler(change2):
    common_filtering(dropdown_sp.value, change2.new)
    
dropdown_sp.observe(dropdown_sp_eventhandler, names='value')
dropdown_taxa.observe(dropdown_taxa_eventhandler, names='value')

display(dropdown_sp)
display(dropdown_taxa)


Dropdown(description='Species: ', options=('Select species', 'Aaptos aaptos', 'Aaptos kanuux', 'Aaptos pernucl…

Dropdown(description='Level of taxonomy to compare: ', options=('Genus', 'Family', 'Order', 'Subclass', 'Class…

In [27]:
display(output)
display(plot_output)

Output()

Output()